In [ ]:
import os
import sys
import numpy as np
import IPython.display

import librosa
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict

# load other modules --> repo root path
sys.path.insert(0, "../")

import torch
from utils import text, audio
from utils import remove_dataparallel_prefix
from utils.logging import Logger
from params.params import Params as hp
from modules.tacotron2 import Tacotron
from dataset.dataset import TextToSpeechDataset, TextToSpeechDatasetCollection, TextToSpeechCollate

In [ ]:
def build_model(checkpoint):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    state = torch.load(checkpoint, map_location=device)
    hp.load_state_dict(state['parameters'])

    model = Tacotron()
    model_dict = model.state_dict()
    pretrained_dict = remove_dataparallel_prefix(state['model'])
    
    missing = [k for k, _ in pretrained_dict.items() if k not in model_dict]
    print(f'Missing model parts: {missing}')
    
    missing = [k for k, _ in model_dict.items() if k not in pretrained_dict]
    print(f'Extra model parts: {missing}')
    
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    
    model_dict.update(pretrained_dict) 
    model.load_state_dict(model_dict) 
    model.to(device)
    
    return model

In [ ]:
def inference(model, inputs, device=None):
    
    inputs = [l.rstrip().split('|') for l in inputs if l]

    spectrograms = []
    for i in inputs:
        
        clean_text = i[0]

        if not hp.use_punctuation: 
            clean_text = text.remove_punctuation(clean_text)
        if not hp.case_sensitive: 
            clean_text = text.to_lower(clean_text)
        if hp.remove_multiple_wspaces: 
            clean_text = text.remove_odd_whitespaces(clean_text)
        
        t = torch.LongTensor(text.to_sequence(clean_text, use_phonemes=hp.use_phonemes))
        
        if hp.multi_language:     
            l_tokens = i[2].split(',')
            t_length = len(clean_text) + 1
            l = []
            for token in l_tokens:
                l_d = token.split('-')
 
                language = [0] * hp.language_number
                for l_cw in l_d[0].split(':'):
                    l_cw_s = l_cw.split('*')
                    language[hp.languages.index(l_cw_s[0])] = 1 if len(l_cw_s) == 1 else float(l_cw_s[1])
                language_length = (int(l_d[1]) if len(l_d) == 2 else t_length)
                l += [language] * language_length
                t_length -= language_length     
            l = torch.FloatTensor([l])
        else:
            l = None

        s = torch.LongTensor([hp.unique_speakers.index(i[1])]) if hp.multi_speaker else None

        if device=="cuda" or (device is None and torch.cuda.is_available()): 
            t = t.cuda(non_blocking=True)
            if l: l = l.cuda(non_blocking=True)
            if s: s = s.cuda(non_blocking=True)
     
        spectrograms.append(model.inference(t, speaker=s, language=l).cpu().detach().numpy())

    return spectrograms

# Synthesis

### Multi

In [ ]:
checkpoint = ""

In [ ]:
torch.load(checkpoint, map_location="cpu")['parameters'];

In [ ]:
hp.decoder_language_init = False

In [ ]:
model = build_model(checkpoint)
model.eval();
print(hp.encoder_type)
print(hp.encoder_dimension)
print(hp.languages)
print(hp.unique_speakers)
print(hp.multi_language)
print(hp.multi_speaker)

In [ ]:
inputs = [
    "jean paul belmondo|00-de|fr",
    "jean paul belmondo|00-de|de*0.1:fr*0.9",
    "jean paul belmondo|00-de|de*0.2:fr*0.8",
    "jean paul belmondo|00-de|de*0.3:fr*0.7",
    "jean paul belmondo|00-de|de*0.4:fr*0.6",
    "jean paul belmondo|00-de|de*0.5:fr*0.5",
    "jean paul belmondo|00-de|de*0.6:fr*0.4",
    "jean paul belmondo|00-de|de*0.7:fr*0.3",
    "jean paul belmondo|00-de|de*0.8:fr*0.2",
    "jean paul belmondo|00-de|de*0.9:fr*0.1",
    "jean paul belmondo|00-de|de",
]

In [ ]:
inputs = []
for s, l in [("00-de","de"),("00-fr","fr"),("00-nl","nl"),("00-zh","zh"),("09-ru","ru")]:
    inputs += [
        f"Das schließlich realisierte projekt vereint die Wien und die Wiener Stadtbahn in einem einheitlichen Profil.|{s}|{l}",
        f"Tijdens de integratie liepen veel klanten weg omdat containers en documenten verdwenen en foutieve rekeningen werden gestuurd.|{s}|{l}",
        f"Совет обладает полномочиями по руководству всей операционной деятельностью Центрального банка и контролю за коммерческими оманскими банками.|{s}|{l}",
        f"C'est l'un des plus beaux palais de Mistretta dont le nom dérive d'une ancienne famille seigneuriale de la ville.|{s}|{l}",
        f"yán zào cì luò， xiānruǎn xiā zhǔ dòufǔ děng shípǐn de pēngdiào jìyì wèi rén suǒ chēngdào， dà cì， bóké mǐ， gāoxiè， zhūtóu zòngděng tǔtèchǎn míngwénxiáěr。|{s}|{l}",
    ]

In [ ]:
inputs = [
    "Charles André Joseph Marie de Gaulle war ein französischer General und Staatsmann.|00-de|fr-36,de",
    "Unter anderem förderte er dank Pierre Brossolette und besonders Jean Moulin die Résistance.|00-de|de-31,fr-18,de-15,fr-11,de-5,fr-10,de",
    "Im April ersetzte de Gaulle den Premierminister Michel Debré durch Georges Pompidou.|00-de|de-18,fr-9,de-21,fr-12,de-7,fr-16,de",
    "Jean Marie Bastien-Thiry war mit dessen Algerien-Politik nicht länger einverstanden.|00-de|fr-24,de",
    "Sein Name wurde auch dem gegenwärtig letzten französischen Flugzeugträger, der Charles de Gaulle gegeben.|00-de|de-79,fr-17,de",
    "Die römisch-katholische Kirche Notre-Dame de Paris ist die Kathedrale des Erzbistums Paris.|00-de|de-31,fr-19,de",
    "Neben Molière oder Balzac gilt er vielen Franzosen als ihr größter Autor überhaupt.|00-de|de-6,fr-7,de-6,fr-6,de",
    "Auf der Geburtsanzeige seines dritten Kindes nennt Hugo sich stolz Baron.|00-de|de-51,fr-4,de",
    "Bei der Abfassung hatte Chateaubriand aber sicher auch opportunistische Motive:|00-de|de-24,fr-13,de",
    "Er musste die Briefe, die er täglich an Juliette Récamier schrieb, diktieren und unterschrieb sie mit zitternder Hand.|00-de|de-40,fr-17,de",
    "Die noch bewohnte Cité de Carcassonne wird von einem doppelten Mauerring umschlossen.|00-de|de-18,fr-19,de",
    "Flussaufwärts sind die nächsten Brücken der Pont de Tancarville und der Pont de Brotonne.|00-de|de-44,fr-19,de-9,fr-16,de",
    "Die Bevölkerung von Marseille war seit jeher stolz und unabhängig und im ganzen Land dafür bekannt,|00-de|de-20,fr-9,de",
    "François Hollande ist ein französischer Politiker der Sozialistischen Partei und war Staatspräsident der Französischen Republik.|00-de|fr-17,de",
    "Sein Amtssitz ist der Élysée-Palast in Paris.|00-de|de-22,fr-13,de",
    "Versailles ist eine französische Stadt in der Region Île-de-France.|00-de|fr-10,de-43,fr-13,de",
    "Die waldreichen Talhänge bilden mit dem Bois des Fonds des Maréchaux.|00-de|de-40,fr-28,de",
    "Zu Beginn seiner Herrschaft hielt sich der Monarch allerdings jeweils nur für kurze Zeit in Versailles auf.|00-de|de-92,fr-10,de",
    "In Versailles benötigten sie häufig nur angemietete Unterkünfte, wovon insbesondere die Gastwirtschaft profitierte.|00-de|de-3,fr-10,de",
    "Nachdem der König der Nationalversammlung jedoch den Zutritt in das Hôtel des Menus Plaisirs verweigert hatte,|00-de|de-68,fr-24,de"
]

In [ ]:
generated_spectrograms = inference(model, inputs, "cpu")

In [ ]:
for i, s in enumerate(generated_spectrograms):
    s = audio.denormalize_spectrogram(s, not hp.predict_linear)
    w = audio.inverse_spectrogram(s, not hp.predict_linear)
    a = IPython.display.Audio(data=w, rate=hp.sample_rate)
    IPython.display.display(a)